In [1]:
import os

os.chdir("..")
os.getcwd()

'd:\\Shaukat ali khan\\programming\\AI-Projects\\langchain-projects\\quiz-generator'

#### Developing vector store for class 9 book physics
To test the topic modeling technique.

In [2]:
from langchain_codebase.codebase import *
from notebooks.modular_code import QuizGenerator
from langchain_core.documents import Document
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
BOOKS_VDB_PATH = "VDB-ALL-BOOKS"

In [5]:
import pandas as pd

def convert_df_to_docs_with_proper_metadata(df: pd.DataFrame):
    """df should have the following keys:
    1. class
    2. subject
    3. page_no
    4. unit_no
    5. content
    Returns docs having content as page_content and other details in the metadata.
    """
    docs = []

    for _, row in df.iterrows():
        class_no = convert_df_val_to_int(row['class'])
        page_no = convert_df_val_to_int(row['page_no'])
        subject = row['subject']
        unit_no = convert_df_val_to_int(row['unit_no'])
        content = row['content']

        doc = Document(page_content=content)
        doc.metadata = {'page': page_no, 'class': class_no, 'subject': subject, 'unit_no': unit_no}
        docs.append(doc)

    return docs

def convert_df_val_to_int(value):
    if isinstance(value, str):
        if value.isnumeric():
            return int(value)
    
    if isinstance(value, float):
        if not pd.isna(value):
            return int(value)
        else:
            return ""
        
    else:
        return value
    


def fetch_relevant_docs_from_vectorstore(vectorstore, topic: str, class_no: int, subject: str, unit_no: int, no_docs: int = 3):
    filters = {
        'class': class_no,
        'subject': subject,
        'unit_no': unit_no
    }
    similar_docs = vectorstore.similarity_search(topic, k=no_docs, filter=filters)
    return similar_docs


In [6]:
dfs_dir = "books_df_csvs"


for index, df_filename in enumerate(os.listdir(dfs_dir)):
    df_path = os.path.join(dfs_dir, df_filename)
    df = QuizGenerator.load_dataframe(df_path)
    docs = convert_df_to_docs_with_proper_metadata(df)

    if not index == 0:
        vectorstore = load_vectorstore(BOOKS_VDB_PATH)
        add_docs_to_chroma_vectorstore(docs, vectorstore)
    else:
        vectorstore = create_vectorstore(docs, BOOKS_VDB_PATH)

    print(f"{df_filename} docs added to {BOOKS_VDB_PATH}")

Loading embeddings and creating vectorstore..
Chemistry 9.csv docs added to VDB-ALL-BOOKS
Loading embeddings and vectorstore..
Computer Sc. 9.csv docs added to VDB-ALL-BOOKS
Loading embeddings and vectorstore..
English 9.csv docs added to VDB-ALL-BOOKS
Loading embeddings and vectorstore..
Physics 9.csv docs added to VDB-ALL-BOOKS
